Packages importeren

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import pyodbc
import sqlite3
import sqlalchemy
import matplotlib.pyplot as plt

Connectie maken met Data Warehouse

In [ ]:
DB = {'servername': 'ENVY-AKRAM\\SQLEXPRESS','database': 'Great Outdoors - Akram en Sylvan'}
conn_str = ('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
export_conn = pyodbc.connect(conn_str, timeout=120)
export_cursor = export_conn.cursor()

engine = sqlalchemy.create_engine(f"mssql+pyodbc:///?odbc_connect={conn_str}")

# Hoe checken we of de connectie werkt?
print(export_cursor.execute("SELECT @@version;"))
'''
DB = {'servername': os.getenv('NAME'),
      'database': os.getenv('DATABASE'),
      'username': os.getenv('USER'),
      'password': os.getenv('PASSWORD')}

# Increase the connection timeout value to 30 seconds
conn_str = f"DRIVER=SQL Server;SERVER={DB['servername']};DATABASE={DB['database']};UID={DB['username']};PWD={DB['password']};Trusted_Connection=yes;Connection"

conn = pyodbc.connect(conn_str, timeout=120)
cursor = conn.cursor()
'''
export_cursor

Connectie maken met brongegevens

In [ ]:
go_sales_connection = sqlite3.connect('Data\\Brongegevens\\go_sales.sqlite')
go_crm_connection = sqlite3.connect('Data\\Brongegevens\\go_crm.sqlite')
go_staff_connection = sqlite3.connect('Data\\Brongegevens\\go_staff.sqlite')
df_inventory = pd.read_csv('Data\\Brongegevens\\GO_SALES_INVENTORY_LEVELSData.csv')
df_product = pd.read_csv('Data\\Brongegevens\\GO_SALES_PRODUCT_FORECASTData.csv')

Helper functie tables inlezen

In [ ]:
def readTable(connection, table):
    query = 'SELECT * FROM ' + table
    result = pd.read_sql_query(query, connection)
    result = result.loc[:, ~result.columns.str.contains('trial', case=False)]
    #verwijder trials
    return result

Database Go Sales

In [ ]:
tables = pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"', go_sales_connection)
country_go_sales = readTable(go_sales_connection, 'country')
order_details = readTable(go_sales_connection, 'order_details')
order_header = readTable(go_sales_connection, 'order_header')
order_method = readTable(go_sales_connection, 'order_method')
product = readTable(go_sales_connection, 'product')
product_line = readTable(go_sales_connection, 'product_line')
product_type = readTable(go_sales_connection, 'product_type')
retailer_site_go_sales = readTable(go_sales_connection, 'retailer_site')
return_reason = readTable(go_sales_connection, 'return_reason')
returned_item = readTable(go_sales_connection, 'returned_item')
sales_branch_go_sales = readTable(go_sales_connection, 'sales_branch')
sales_staff_go_sales = readTable(go_sales_connection, 'sales_staff')
sales_targetdata = readTable(go_sales_connection, 'SALES_TARGETData')
tables  


Database Go Staff

In [ ]:
tables = pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"', go_staff_connection)
course = readTable(go_staff_connection, 'course')
sales_branch_go_staff = readTable(go_staff_connection, 'sales_branch')
sales_staff_go_staff = readTable(go_staff_connection, 'sales_staff')
satisfaction = readTable(go_staff_connection, 'satisfaction')
satisfaction_type = readTable(go_staff_connection, 'satisfaction_type')
training = readTable(go_staff_connection, 'training')
tables


Database Go Crm

In [ ]:
tables = pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"', go_crm_connection)
age_group = readTable(go_crm_connection, 'age_group')
country_go_crm = readTable(go_crm_connection, 'country')
retailer = readTable(go_crm_connection, 'retailer')
retailer_contact = readTable(go_crm_connection, 'retailer_contact')
retailer_headquarters = readTable(go_crm_connection, 'retailer_headquarters')
retailer_segment = readTable(go_crm_connection, 'retailer_segment')
retailer_site_go_crm = readTable(go_crm_connection, 'retailer_site')
retailer_type = readTable(go_crm_connection, 'retailer_type')
sales_demographic = readTable(go_crm_connection, 'sales_demographic')
sales_territory = readTable(go_crm_connection, 'sales_territory')
tables

retailer_headquarters


# Unions

In [ ]:
sales_staff_union = pd.concat([sales_staff_go_staff, sales_staff_go_sales])
sales_staff_union.drop_duplicates(subset='SALES_STAFF_CODE', ignore_index=True, inplace=True)
sales_staff_union
#heeft overwegingen nodig op: 
#manager_code


In [ ]:
sales_branch_union = pd.concat([sales_branch_go_staff, sales_branch_go_sales])
sales_branch_union.drop_duplicates(subset='SALES_BRANCH_CODE', ignore_index=True, inplace=True)
sales_branch_union

In [ ]:
retailer_site_union = pd.concat([retailer_site_go_crm, retailer_site_go_sales])
retailer_site_union.drop_duplicates(subset='RETAILER_SITE_CODE', ignore_index=True, inplace=True)
retailer_site_union

In [ ]:
country_go_crm_union = pd.merge(country_go_crm, sales_territory, on='SALES_TERRITORY_CODE', how='outer')
country_union = pd.merge(country_go_sales, country_go_crm_union, on='COUNTRY_CODE', how='outer')
country_union.drop_duplicates(subset='COUNTRY_CODE', ignore_index=True, inplace=True)
country_union
#not actually a union

# Verborgen Dimensies

Date

In [ ]:
date_dict = {'DATE_DATE':[], 'DATE_YEAR':[], 'DATE_QUARTER':[], 'DATE_MONTH':[], 'DATE_DAY':[]}

date_dw = pd.DataFrame(date_dict)
date_dw

ToDate() Functions

In [ ]:
#??

Month

In [ ]:
#iets

# Simpele Dimensies

Return Reason

In [ ]:
def ToReasonCategory(str):

    if 'wrong' in str.lower():
        return 'Wrong Product'
    elif 'unsatisfactory' in str.lower():
        return 'Unsatisfactory'
    else:
        return 'Faulty'

return_reason_dw = return_reason.copy()
return_reason_dw['RETURN_CATEGORY'] = return_reason_dw['RETURN_DESCRIPTION_EN'].apply(ToReasonCategory)
return_reason_dw

#trial888

Course

In [ ]:
def ToCourseCategory(str):

    if 'marketing' in str.lower():
        return 'Marketing'
    elif 'sales' in str.lower():
        return 'Sales'
    elif 'management' in str.lower():
        return 'Management'
    elif 'communication' in str.lower():
        return 'Communication'
    elif 'orientation' in str.lower():
        return 'Orientation'
    else: 
        return 'Other'

course_dw = course.copy()
course_dw['COURSE_CATEGORY'] = course_dw['COURSE_DESCRIPTION'].apply(ToCourseCategory)
course_dw

#trial633

Order Method

In [ ]:
order_method_dw = order_method.copy()
#fuck de categorie

Satisfaction Type

In [ ]:
def ToSatisfied(nr):
    try:
        return int(nr) >= 3
    except ValueError:
        return False

satisfaction_type_dw = satisfaction_type.copy()
satisfaction_type_dw['SATISFIED_BOOLEAN'] = satisfaction_type['SATISFACTION_TYPE_CODE'].apply(ToSatisfied)
satisfaction_type_dw

# Merges (dimensies)

Retailer Headquarters

In [ ]:
retailer_headquarters_dw = pd.merge(
    retailer_headquarters.add_prefix('RT_HEADQUARTERS_'), country_union.add_prefix('COUNTRY_'), 
    left_on = 'RT_HEADQUARTERS_COUNTRY_CODE', right_on='COUNTRY_COUNTRY_CODE', how='outer')
retailer_headquarters_dw = pd.merge(
    retailer_headquarters_dw, retailer_segment.add_prefix('RT_SEGMENT_'), 
    left_on = 'RT_HEADQUARTERS_SEGMENT_CODE', right_on='RT_SEGMENT_SEGMENT_CODE', how='outer')
#segment category
retailer_headquarters_dw

Retailer Contact

In [ ]:
retailer_contact_dw = pd.merge(
    retailer_contact.add_prefix('RT_CONTACT_'), retailer_site_union.add_prefix('RT_SITE_'), 
    left_on='RT_CONTACT_RETAILER_SITE_CODE', right_on='RT_SITE_RETAILER_SITE_CODE', how='inner')
#position category
retailer_contact_dw = pd.merge(
    retailer_contact_dw, retailer.add_prefix('RT_'), 
    left_on='RT_SITE_RETAILER_CODE', right_on='RT_RETAILER_CODE', how='inner')
retailer_contact_dw = pd.merge(
    retailer_contact_dw, retailer_type.add_prefix('RT_TYPE_'), 
    left_on='RT_RETAILER_TYPE_CODE', right_on='RT_TYPE_RETAILER_TYPE_CODE', how='inner')
retailer_contact_dw = pd.merge(
    retailer_contact_dw, retailer_headquarters_dw, 
    left_on='RT_RETAILER_CODEMR', right_on='RT_HEADQUARTERS_RETAILER_CODEMR', how='inner')
retailer_contact_dw
#trial222/888
#loc: any non americas?

Sales Staff

In [ ]:
sales_staff_dw = pd.merge(
    sales_staff_union.add_prefix('SALES_STAFF_'), sales_branch_union.add_prefix('SALES_BRANCH_'), 
    left_on='SALES_STAFF_SALES_BRANCH_CODE', right_on='SALES_BRANCH_SALES_BRANCH_CODE', how='inner')
sales_staff_dw = pd.merge(
    sales_staff_dw, country_go_sales.add_prefix('COUNTRY_'),
    left_on='SALES_BRANCH_COUNTRY_CODE', right_on='COUNTRY_COUNTRY_CODE', how='inner')
sales_staff_dw
#trial633/888

Retailer Headquarters

In [ ]:
#wat doen we hiermee?
"""
retailer_headquarters = pd.merge(
    retailer_headquarters[['RETAILER_CODEMR', 'RETAILER_NAME', 'ADDRESS1', 'ADDRESS2', 'POSTAL_ZONE', 'CITY', 'COUNTRY_CODE', 'REGION', 'PHONE', 'FAX', 'SEGMENT_CODE']], 
    country[['COUNTRY_CODE', 'COUNTRY_EN', 'FLAG_IMAGE', 'SALES_TERRITORY_CODE']], left_on='COUNTRY_CODE', right_on='COUNTRY_CODE', 
    how='inner'
    ).merge(sales_territory[['SALES_TERRITORY_CODE', 'TERRITORY_NAME_EN']], left_on='SALES_TERRITORY_CODE', right_on='SALES_TERRITORY_CODE', how='inner'
            ).merge(retailer_segment[['SEGMENT_CODE', 'LANGUAGE', 'SEGMENT_NAME', 'SEGMENT_DESCRIPTION']], left_on='SEGMENT_CODE', right_on='SEGMENT_CODE', how='inner')
retailer_headquarters
"""

Product

In [ ]:
#zijn de koloomen nodig?
#prefixes toevoegen
product_dw = pd.merge(
    product[['PRODUCT_NUMBER', 'PRODUCT_NAME', 'PRODUCT_IMAGE', 'DESCRIPTION', 'INTRODUCTION_DATE', 'PRODUCTION_COST', 'MARGIN', 'LANGUAGE', 'PRODUCT_TYPE_CODE']], 
    product_type[['PRODUCT_TYPE_CODE', 'PRODUCT_TYPE_EN', 'PRODUCT_LINE_CODE']], 
    left_on='PRODUCT_TYPE_CODE', right_on='PRODUCT_TYPE_CODE', how='inner'
    ).merge( 
        product_line[['PRODUCT_LINE_CODE', 'PRODUCT_LINE_EN']], 
        left_on='PRODUCT_LINE_CODE', right_on='PRODUCT_LINE_CODE', how='inner')
product_dw

Sales Demographic

In [ ]:
#prefixes
sales_demographic = pd.merge(
    sales_demographic[['DEMOGRAPHIC_CODE', 'SALES_PERCENT', 'RETAILER_CODEMR', 'AGE_GROUP_CODE']]
    , age_group[['AGE_GROUP_CODE', 'UPPER_AGE', 'LOWER_AGE']], left_on='AGE_GROUP_CODE', right_on='AGE_GROUP_CODE', how='inner')
sales_demographic

# Merges (feiten)

Sales Target

In [ ]:
sales_targetdata_dw = pd.DataFrame()

sales_targetdata_dw['SALES_TARGETDATA_CODE'] = sales_targetdata['Id']
sales_targetdata_dw['SALES_TARGETDATA_SALES_TARGET'] = sales_targetdata['SALES_TARGET']
sales_targetdata_dw['MONTH_YEAR'] = sales_targetdata['SALES_PERIOD']
sales_targetdata_dw['MONTH_MONTH'] = sales_targetdata['SALES_YEAR']
sales_targetdata_dw['PRODUCT_PRODUCT_NUMBER'] = sales_targetdata['PRODUCT_NUMBER']
sales_targetdata_dw['SALES_STAFF_SALES_STAFF_CODE'] = sales_targetdata['SALES_STAFF_CODE']
sales_targetdata_dw['RETAILER_RETAILER_CODE'] = sales_targetdata['RETAILER_CODE']

sales_targetdata_dw
#take month from MONTH
#ETL klopt niet - sales branch en order method verbinden niet

#trial879

Inventory Levels

In [ ]:
inventory_levels_dw = pd.DataFrame()

#inventory_count is leeg?
inventory_levels_dw['PRODUCT_NUMBER'] = df_inventory['PRODUCT_NUMBER']
inventory_levels_dw['INVENTORY_YEAR'] = df_inventory['INVENTORY_YEAR']
inventory_levels_dw['INVENTORY_MONTH'] = df_inventory['INVENTORY_MONTH']
inventory_levels_dw['INVENTORY_COUNT'] = df_inventory['INVENTORY_COUNT']

inventory_levels_dw

Training

In [ ]:
#prefixes
training = pd.merge(
    course[['COURSE_CODE', 'COURSE_DESCRIPTION']],
    training[['COURSE_CODE', 'SALES_STAFF_CODE', 'YEAR']], left_on='COURSE_CODE', right_on='COURSE_CODE', how='inner')
training

Satisfaction

In [ ]:
#prefixes
satisfaction = pd.merge(
    satisfaction[['YEAR', 'SALES_STAFF_CODE', 'SATISFACTION_TYPE_CODE']],
    satisfaction_type[['SATISFACTION_TYPE_CODE', 'SATISFACTION_TYPE_DESCRIPTION']], left_on='SATISFACTION_TYPE_CODE', right_on='SATISFACTION_TYPE_CODE', how='inner')
satisfaction

Order

In [ ]:
def ToOrderTurnover(quantity, sale_price):
    return float(quantity) * float(sale_price)

def ToOrderProfit(quantity, sale_price, unit_cost):
    return float(quantity) * (float(sale_price) - float(unit_cost))

In [ ]:
order_dw = pd.merge(
    order_header,
    order_details,
    on='ORDER_NUMBER', how='inner'
)
order_dw = order_dw.drop('RETAILER_NAME', axis=1)

order_dw['ORDER_TURNOVER'] = order_dw.apply(lambda x: ToOrderTurnover(x['QUANTITY'], x['UNIT_SALE_PRICE']), axis = 1)
order_dw['ORDER_PROFIT'] = order_dw.apply(lambda x: ToOrderProfit(x['QUANTITY'], x['UNIT_SALE_PRICE'], x['UNIT_COST']), axis = 1)

order_dw

Returned Item

In [ ]:
def ToTurnunder (quantity, sale_price):
    return float(quantity) * float(sale_price)

def ToReturnPercentage(quantity, return_quantity):
    return 100 * (float(return_quantity) / float(quantity))

In [ ]:
returned_item_dw = pd.merge(
    order_details[['ORDER_DETAIL_CODE', 'QUANTITY', 'UNIT_SALE_PRICE']],
    returned_item,
    on='ORDER_DETAIL_CODE', how='inner'
)

returned_item_dw['TURNUNDER'] = returned_item_dw.apply(
    lambda x: ToTurnunder(x['RETURN_QUANTITY'], x['UNIT_SALE_PRICE']), 
    axis = 1)
returned_item_dw['RETURN_PERCENTAGE'] = returned_item_dw.apply(
    lambda x: ToReturnPercentage(x['QUANTITY'], x['RETURN_QUANTITY']),
    axis = 1)

returned_item_dw

Product Forecast

In [ ]:
def ForecastCost(quantity, unit_cost):
    return float(quantity) * float(unit_cost)


df_product_dw = pd.merge(
    df_product,
    product[['PRODUCT_NUMBER', 'PRODUCTION_COST']],
    left_on=df_product['PRODUCT_NUMBER'].astype(str),
    right_on='PRODUCT_NUMBER', 
    how='inner'
)

#onelegant - zou anders moeten kunnen
df_product_dw = df_product_dw.drop('PRODUCT_NUMBER_x', axis=1)
df_product_dw = df_product_dw.drop('PRODUCT_NUMBER_y', axis=1)

df_product_dw['FORECAST_COST'] = df_product_dw.apply(
    lambda x: ForecastCost(x['EXPECTED_VOLUME'], x['PRODUCTION_COST']), 
    axis = 1)

df_product_dw

# Naar Database pushen

To SQL

In [ ]:
return_reason_dw = return_reason_dw.astype(dict)
return_reason_dw.dtypes
return_reason_dw

In [ ]:
return_reason_dw.to_sql('RETURN_REASON', engine, if_exists='replace', index=False)
course_dw.to_sql('COURSE', engine, if_exists='replace', index=False)
order_method_dw.to_sql('ORDER_METHOD', engine, if_exists='replace', index=False)
retailer_contact_dw.to_sql('RETAILER_CONTACT', engine, if_exists='replace', index=False)
retailer_headquarters_dw.to_sql('RETAILER_HEADQUARTERS', engine, if_exists='replace', index=False)
sales_staff_dw.to_sql('SALES_STAFF', engine, if_exists='replace', index=False)
sales_targetdata_dw.to_sql('SALES_TARGETDATA', engine, if_exists='replace', index=False)
satisfaction_dw.to_sql('SATISFACTION', engine, if_exists='replace', index=False)
satisfaction_type_dw.to_sql('SATISFACTION_TYPE', engine, if_exists='replace', index=False)
training_dw.to_sql('TRAINING', engine, if_exists='replace', index=False)
sales_demographic_dw.to_sql('SALES_DEMOGRAPHIC', engine, if_exists='replace', index=False)
product_dw.to_sql('PRODUCT', engine, if_exists='replace', index=False)

# Regressiemodel

In [ ]:
from datetime import datetime
#from sklearn.model_selection import train_test_split
#from sklearn import linear_model
#from sklearn.linear_model import LinearRegression
#from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

Gegevens ophalen uit Data Warehouse

In [ ]:
#->>

Dataframe opstellen

In [78]:
datasetPrepProduct = pd.merge(
    order_dw[['PRODUCT_NUMBER', 'QUANTITY', 'UNIT_COST']],
    product_dw[['PRODUCT_NUMBER', 'INTRODUCTION_DATE', 'PRODUCT_TYPE_CODE', 'PRODUCT_LINE_CODE']],
    on='PRODUCT_NUMBER', how='inner'
)
datasetPrepProduct['UNIT_COST'] = datasetPrepProduct['UNIT_COST'].astype(float)
datasetPrepProduct

,PRODUCT_NUMBER,QUANTITY,UNIT_COST,INTRODUCTION_DATE,PRODUCT_TYPE_CODE,PRODUCT_LINE_CODE
0,2,100,9.64,15-2-2011,1,1
1,32,254,18.25,15-2-2011,5,1
2,35,114,19.67,26-10-2016,5,1
3,36,130,21.03,15-2-2011,5,1
4,39,74,42.33,5-3-2013,5,1
...,...,...,...,...,...,...
43058,52,64,19.00,10-1-2021,8,2
43059,56,24,9.00,10-1-2021,8,2
43060,57,40,40.00,10-1-2021,9,2
43061,78,32,79.19,15-2-2011,13,3


In [79]:
#helperfunctie

def ToProductAge(date_str):
    date_obj = pd.to_datetime(date_str, dayfirst=True)
    return (datetime.now() - date_obj).days


In [80]:
datasetProduct = datasetPrepProduct.groupby('PRODUCT_NUMBER').agg({
    'QUANTITY': 'sum',
    'INTRODUCTION_DATE': 'first',
    'UNIT_COST': 'mean',
    'PRODUCT_TYPE_CODE': 'first',
    'PRODUCT_LINE_CODE': 'first'
}).reset_index()
datasetProduct

,PRODUCT_NUMBER,QUANTITY,INTRODUCTION_DATE,UNIT_COST,PRODUCT_TYPE_CODE,PRODUCT_LINE_CODE
0,1,41770,15-2-2011,4.113607,1,1
1,10,15928,15-2-2011,10.429155,1,1
2,100,29980,15-2-2011,2.848523,17,4
3,101,5764,15-12-2019,316.939286,18,5
4,102,5014,10-12-2019,402.653541,18,5
...,...,...,...,...,...,...
110,95,53086,15-2-2011,2.875822,16,4
111,96,5974,15-2-2011,16.269130,17,4
112,97,3246,5-3-2013,24.459522,17,4
113,98,18362,15-2-2011,2.897125,17,4


Variabelen numeriek maken

In [81]:
datasetProduct

,PRODUCT_NUMBER,QUANTITY,INTRODUCTION_DATE,UNIT_COST,PRODUCT_TYPE_CODE,PRODUCT_LINE_CODE
0,1,41770,15-2-2011,4.113607,1,1
1,10,15928,15-2-2011,10.429155,1,1
2,100,29980,15-2-2011,2.848523,17,4
3,101,5764,15-12-2019,316.939286,18,5
4,102,5014,10-12-2019,402.653541,18,5
...,...,...,...,...,...,...
110,95,53086,15-2-2011,2.875822,16,4
111,96,5974,15-2-2011,16.269130,17,4
112,97,3246,5-3-2013,24.459522,17,4
113,98,18362,15-2-2011,2.897125,17,4


In [82]:
datasetProduct['DAYS_SINCE_INTRODUCTION'] = datasetProduct['INTRODUCTION_DATE'].apply(ToProductAge)
datasetProduct.drop('INTRODUCTION_DATE', axis=1, inplace=True)


typeDummies = pd.get_dummies(datasetProduct['PRODUCT_TYPE_CODE'])
datasetProduct = pd.concat([datasetProduct, typeDummies], axis=1)
datasetProduct.drop('PRODUCT_TYPE_CODE', axis=1, inplace=True)

lineDummies = pd.get_dummies(datasetProduct['PRODUCT_LINE_CODE'])
datasetProduct = pd.concat([datasetProduct, lineDummies], axis = 1)
datasetProduct.drop('PRODUCT_LINE_CODE', axis=1, inplace=True)


datasetProduct

,PRODUCT_NUMBER,QUANTITY,UNIT_COST,DAYS_SINCE_INTRODUCTION,1,10,11,12,13,14,...,5,6,7,8,9,1,2,3,4,5
0,1,41770,4.113607,4800,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,10,15928,10.429155,4800,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,100,29980,2.848523,4800,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,101,5764,316.939286,1575,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,102,5014,402.653541,1580,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,95,53086,2.875822,4800,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
111,96,5974,16.269130,4800,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
112,97,3246,24.459522,4051,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
113,98,18362,2.897125,4800,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


Dataset opdelen

In [83]:
x = datasetProduct.drop('QUANTITY', axis = 1)
y = datasetProduct.loc[:,['QUANTITY']]
y

,QUANTITY
0,41770
1,15928
2,29980
3,5764
4,5014
...,...
110,53086
111,5974
112,3246
113,18362


In [84]:
#horizontaal en verticaal snijden
x_train, x_test, y_train, y_test = train_test_split(
    x, y, 
    test_size=0.15, random_state=42
)

NameError: name 'train_test_split' is not defined

In [ ]:
#regressiemodel bouwen
model = LinearRegression().fit(x_train, y_train)
model


In [ ]:
#predictions maken
y_pred = model.predict(x_test)
y_df = pd.DataFrame(y_pred, columns=['QUANTITY_PREDICTED'])
y_df

In [ ]:
#combineren
y_test_predictions_merge = pd.concat([y_test.reset_index()['QUANTITY'], y_df], axis=1)
y_test_predictions_merge.loc[y_test_predictions_merge['QUANTITY_PREDICTED'].notna(),:]

Resultaten analyseren

In [ ]:
mean_squared_error(y_test['QUANTITY'], y_pred)

In [ ]:
mean_absolute_error(y_test['QUANTITY'], y_pred)

# Classificatiemodel

In [ ]:
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

# Data inlezen en kolommen selecteren

In [74]:
df = pd.read_csv("titanic.csv", sep = ';')
df

FileNotFoundError: [Errno 2] No such file or directory: 'titanic.csv'

In [76]:
df = df.loc[:, ['Pclass', 'Sex', 'Age', 'Survived']]	

NameError: name 'df' is not defined

In [77]:
df['Pclass'] = df['Pclass'].astype('str')
df.dtypes

NameError: name 'df' is not defined

In [ ]:
dummies_dataframe = pd.get_dummies(df.loc[:, ['Sex', 'Pclass']])
dummies_dataframe

In [ ]:
df = pd.concat

# Vanaf hier kan je het gebruiken


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size = 0.15,
    random_state=42
)

In [ ]:
list(x.columns)

# Decision Tree bouwen en evalueren 

Max_depth = 2

In [ ]:
dtree = DecisionTreeClassifier(max_depth = 2)
dtree = dtree.fit(x_train, y_train)
tree.plot_tree(dtree, feature_names, x.columns)
plt.show()

In [ ]:
df_predicted = pd.Dataframe(dtree.predict(x_test))
df_predicted = df_predicted.rename(columns = {0: 'Predicted_Survived'})
resultaat_model = pd.concat([y_test.reset_index()['Survived'], predicted_df], axis = 1)
resultaat_model

In [ ]:
cf_matrix = metrics.cf_matrix(resultaat_model['Survived'], resultaat_model['Predicted_Survived'])
cf_matrix_display = metrics.ConfusionMatrixDisplay(cf_matrix, display_labels = [False, True])

cf_matrix_display.plot()
plt.show()

In [ ]:
metrics.accuracy_score(resultaat_model['Survived'], resultaat_model['Predicted_Survived'])

Max_depth = 3

In [ ]:
dtree = DecisionTreeClassifier(max_depth = 3)
dtree = dtree.fit(x_train, y_train)
tree.plot_tree(dtree, feature_names, x.columns)
plt.show()

In [ ]:
df_predicted = pd.Dataframe(dtree.predict(x_test))
df_predicted = df_predicted.rename(columns = {0: 'Predicted_Survived'})
resultaat_model = pd.concat([y_test.reset_index()['Survived'], predicted_df], axis = 1)
resultaat_model

In [ ]:
cf_matrix = metrics.cf_matrix(resultaat_model['Survived'], resultaat_model['Predicted_Survived'])
cf_matrix_display = metrics.ConfusionMatrixDisplay(cf_matrix, display_labels = [False, True])

cf_matrix_display.plot()
plt.show()

In [ ]:
metrics.accuracy_score(resultaat_model['Survived'], resultaat_model['Predicted_Survived'])

Geen Max_depth

In [ ]:
dtree = DecisionTreeClassifier()
dtree = dtree.fit(x_train, y_train)
tree.plot_tree(dtree, feature_names, x.columns)
plt.show()

Evalueren

In [ ]:
df_predicted = pd.Dataframe(dtree.predict(x_test))
df_predicted = df_predicted.rename(columns = {0: 'Predicted_Survived'})
resultaat_model = pd.concat([y_test.reset_index()['Survived'], predicted_df], axis = 1)
resultaat_model

In [ ]:
cf_matrix = metrics.cf_matrix(resultaat_model['Survived'], resultaat_model['Predicted_Survived'])
cf_matrix_display = metrics.ConfusionMatrixDisplay(cf_matrix, display_labels = [False, True])

cf_matrix_display.plot()
plt.show()

In [ ]:
metrics.accuracy_score(resultaat_model['Survived'], resultaat_model['Predicted_Survived'])